In [18]:
import gurobipy as gp
from gurobipy import Model, GRB

## Given ##
# Period Tasks: (period/deadline, execution time)
# T1 = (20,5)
# T2 = (25,9)
# T3 = (100,5)
# T4 = (100,10)

# Hyperperiod = 100

#* Frame Size = 10

In [20]:
# Given:
hyperperiod = 100
frameSize = 10

tasks = ['J1','J2','J3',"J4"]
period = {'J1': 20,  # Task 1
          'J2': 25,  # Task 2
          'J3': 100, # Task 3
          'J4': 100  # Task 4
          }
execution_time = {"J1": 5, # Task 1
                  "J2": 9, # Task 2
                  "J3": 5, # Task 3
                  "J4": 10 # Task 4
                  }

## Given ##
# Period Tasks: (period/deadline, execution time)
# T1 = (30,10)
# T2 = (60,15)
# T3 = (90,20)

# Hyperperiod = 180

# Frame Size = 30

In [21]:
# # Given:
# hyperperiod = 180
# frameSize = 30

# tasks = ['J1','J2','J3']
# period = {'J1': 30,  # Task 1
#           'J2': 60,  # Task 2
#           'J3': 90, # Task 3
#           }
# execution_time = {"J1": 10, # Task 1
#                   "J2": 15, # Task 2
#                   "J3": 20, # Task 3
#                   }

In [22]:
### Frame Based Scheduling ###

## List of jobs:
jobs = []
for task in tasks:
    for i in range(1, int(hyperperiod/period[task])+ 1):
        jobs.append( f"{task}{i}" )

## List of Frames:
frames = []
for i in range(1, int(hyperperiod/frameSize) + 1):
    frames.append(f"F{i}")


### Frame Start Times:
frameStartTimes = dict()
for i in range( int(hyperperiod/frameSize) ):
    frameStartTimes[frames[i]] = int(i * frameSize)

#### Release, Deadlines, Execution times

releaseTimes = dict()
for task in tasks:
    for i in range(1, int(hyperperiod/period[task]) + 1):
        releaseTimes[ task+str(i) ] = (i - 1) * period[task]

### Deadlines of all jobs:

deadlines = dict()
for task in tasks:
    for i in range(1, int(hyperperiod/period[task]) + 1):
        deadlines[ task+str(i) ] = i * period[task]

### Execution Times of all jobs:

execution_times = dict()
for job in jobs:
    execution_times[job] = execution_time[job[0:2]]

#### Set of Eligible Frames for each job:
# Frame start must be >= job release time
# Frame end must be <= job deadline
eligibleFrames = dict()

for job in jobs:
    for frame,frameStartTime in frameStartTimes.items():
        eligibleFrames[ (job,frame) ] = 1 if (releaseTimes[job] <= frameStartTime) and (deadlines[job] >= frameStartTime + frameSize) else -1

In [23]:
# Create Gurobi model
m = Model()

In [24]:
# Binary Variables for Frame Assignment:
x = m.addVars(jobs, frames, vtype=GRB.BINARY, name="x")

In [25]:
# Constraint: assign exactly one frame to each job
for job in jobs:
    m.addConstr(sum(x[job, frame] for frame in frames) == 1, name=f"assign_{job}")

# Constraint: only assign jobs to eligible Frames:
for job in jobs:
    for frame in frames:
        m.addConstr( x[job,frame] * eligibleFrames[(job,frame)] >= 0)

# Constraint: do not exceed the capacity of the Frame:
for frame in frames:
    m.addConstr( sum(x[job,frame]*execution_times[job] for job in jobs) <= frameSize )

In [26]:
# Add Objective: Maximize the least slack time:
leastSlackTme = m.addVar(vtype = GRB.INTEGER, name = 'leastSlackTme')

for frame in frames:
    m.addConstr( frameSize - sum( x[job,frame]*execution_times[job] for job in jobs) >= leastSlackTme)

m.setObjective(leastSlackTme, GRB.MAXIMIZE)

In [27]:
# Disable presolve
m.setParam("Presolve", 0)

# Optimize the model
m.optimize()

Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11370H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 89 rows, 67 columns and 270 nonzeros
Model fingerprint: 0xca7b7d24
Variable types: 0 continuous, 67 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -0.0000000
Variable types: 0 continuous, 67 integer (66 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best boun

In [28]:
# Output the results
if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for job in jobs:
        for frame in frames:
            if x[job,frame].X:
                print(f"{job}: Assigned to Frame -> {frame}")   
else:
    print("No optimal solution found.")

print("\nSlack Times:")
# Output slack time
for frame in frames:
    print(f"Frame {frame}:" , frameSize - sum(x[job,frame].X*execution_times[job] for job in jobs))

print("\nLeast Slack Time =", leastSlackTme.X)

Optimal solution found!
J11: Assigned to Frame -> F1
J12: Assigned to Frame -> F2
J13: Assigned to Frame -> F3
J14: Assigned to Frame -> F4
J15: Assigned to Frame -> F5
J16: Assigned to Frame -> F6
J21: Assigned to Frame -> F2
J22: Assigned to Frame -> F4
J23: Assigned to Frame -> F5
J31: Assigned to Frame -> F1
J32: Assigned to Frame -> F6

Slack Times:
Frame F1: 0.0
Frame F2: 5.0
Frame F3: 20.0
Frame F4: 5.0
Frame F5: 5.0
Frame F6: 0.0

Least Slack Time = 0.0
